In [1]:
import pandas as pd
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import random
import string
import re
#tokenize and remove stopword
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
from nltk.stem.snowball import SnowballStemmer

In [2]:
with open('A:/PWR/project/SMA/News_Category_Dataset_v2.json') as f:
    data = pd.DataFrame(json.loads(line) for line in f)
pd.set_option('max_colwidth', 1000)    
data

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89,She left her husband. He killed their children. Just another day in America.
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c,The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675,The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,https://www.huffingtonpost.com/entry/julianna-margulies-trump-poop-bag_us_5b093ec2e4b0fdb2aa53df70,"The ""Dietland"" actress said using the bags is a ""really cathartic, therapeutic moment."""
5,Ron Dicker,ENTERTAINMENT,2018-05-26,Morgan Freeman 'Devastated' That Sexual Harassment Claims Could Undermine Legacy,https://www.huffingtonpost.com/entry/morgan-freeman-devastated-sexual-misconduct_us_5b096319e4b0802d69cba298,"""It is not right to equate horrific incidents of sexual assault with misplaced compliments or humor,"" he said in a statement."
6,Ron Dicker,ENTERTAINMENT,2018-05-26,Donald Trump Is Lovin' New McDonald's Jingle In 'Tonight Show' Bit,https://www.huffingtonpost.com/entry/donald-trump-mcondalds-tonight-show_us_5b093561e4b0fdb2aa53daba,"It's catchy, all right."
7,Todd Van Luling,ENTERTAINMENT,2018-05-26,What To Watch On Amazon Prime That’s New This Week,https://www.huffingtonpost.com/entry/amazon-prime-what-to-watch_us_5b044625e4b0c0b8b23ec14f,There's a great mini-series joining this week.
8,Andy McDonald,ENTERTAINMENT,2018-05-26,Mike Myers Reveals He'd 'Like To' Do A Fourth Austin Powers Film,https://www.huffingtonpost.com/entry/mike-myers-reveals-he-wants-to-do-a-fourth-austin-powers-film_us_5b096198e4b0802d69cb9f15,"Myer's kids may be pushing for a new ""Powers"" film more than anyone."
9,Todd Van Luling,ENTERTAINMENT,2018-05-26,What To Watch On Hulu That’s New This Week,https://www.huffingtonpost.com/entry/hulu-what-to-watch_us_5b0445bae4b0c0b8b23ec046,You're getting a recent Academy Award-winning movie.


In [3]:
def removeNonAscii(s): 
    return "".join(i for i in s if ord(i)<128)

In [4]:
author=[]
for a in data['authors']:
    auth=removeNonAscii(a.lower())
    author.append(auth)

categ=[]
for b in data['category']:
    cat=removeNonAscii(b.lower())
    categ.append(cat)
    
headlines=[]
for c in data['headline']:
    hl=removeNonAscii(c.lower())
    headlines.append(hl)
    
short=[]
for d in data['short_description']:
    sd=removeNonAscii(d.lower())
    short.append(sd)

In [5]:
clean = pd.DataFrame(dict(authors=author, category=categ, headline=headlines, short_description=short)) 

In [6]:
#clean.to_csv('A:/PWR/project/SMA/dataclean.csv')

In [3]:
dataset = pd.read_csv('A:/PWR/project/SMA/data.csv')
dataset

,id,category,headline,short_desc
0,1,business,"u.s. launches auto import probe, china vows to defend its interests",the investigation could lead to new u.s. tariffs similar to those imposed on imported steel and aluminum in march.
1,2,business,starbucks says anyone can now sit in its cafes -- even without buying anything,the new policy was unveiled weeks after the controversial arrest of two black men at a philadelphia starbucks.
2,3,business,seattle passes controversial new tax on city's biggest companies to combat housing crisis,"following the council vote, amazons vice president, drew herdener, said the company has resumed construction planning for its so-called block 18 project in downtown seattle."
3,4,business,"uber ends forced arbitration in individual cases of sexual assault, harassment",victims will be free to go to court -- but a few caveats remain.
4,5,business,"chili's hit by data breach, credit and debit card information compromised",the breach is believed to have occurred between march and april.
5,6,business,how uber silences women after sexual assaults,"just by using the app, you click away your rights. now some victims are fighting for change."
6,7,business,how amazon is holding seattle hostage,"the city wants to tax large corporations to pay for homeless housing, but jeff bezos isn't pleased."
7,8,business,bank of america appears to flip on firearm promise with loan to remington,the bank pledged in april to stop financing companies that sell military-style firearms to civilians. remington made the bushmaster rifle used in the sandy hook school massacre.
8,9,business,ex-volkswagen ceo charged in u.s. over emissions cheating scandal,martin winterkorn resigned soon after the scandal became public in 2015.
9,10,business,"women describe rampant groping, sexual harassment at verizon-contracted warehouse",verizon launched an investigation soon after huffpost reached out for comment.


### PREPROCESSING

In [5]:
#PREPROSESSING
def strip_links(text):
    link_regex    = re.compile("([^-_a-zA-Z0-9!@#%&=,/'\";:~`\$\^\*\(\)\+\[\]\.\{\}\|\?\<\>\\]+|[^\s]+)")
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def drop_digits(in_str):
    digit_list = "1234567890"
    for char in digit_list:
        in_str = in_str.replace(char, "")
    return in_str

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def stemmers(text):
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    # Return a list of words
    return(text)

In [6]:
headlines2=[]
for c2 in dataset['headline']:
    hl2=strip_all_entities(drop_digits(strip_links(c2)))
    headlines2.append(hl2)

In [7]:
short2=[]
for d2 in dataset['short_desc']:
    sd2=stemmers(strip_all_entities(drop_digits(strip_links(d2))))
    short2.append(sd2)

In [8]:
preproses = pd.DataFrame(dict(category=dataset.category, headline=headlines2, short_desc=short2)) 
preproses


,category,headline,short_desc
0,business,u s launches auto import probe china vows to defend its interests,the investig could lead to new u s tariff similar to those impos on import steel and aluminum in march
1,business,starbucks says anyone can now sit in its cafes even without buying anything,the new polici was unveil week after the controversi arrest of two black men at a philadelphia starbuck
2,business,seattle passes controversial new tax on city s biggest companies to combat housing crisis,follow the council vote amazon vice presid drew herden said the compani has resum construct plan for it so call block project in downtown seattl
3,business,uber ends forced arbitration in individual cases of sexual assault harassment,victim will be free to go to court but a few caveat remain
4,business,chili s hit by data breach credit and debit card information compromised,the breach is believ to have occur between march and april
5,business,how uber silences women after sexual assaults,just by use the app you click away your right now some victim are fight for chang
6,business,how amazon is holding seattle hostage,the citi want to tax larg corpor to pay for homeless hous but jeff bezo isn t pleas
7,business,bank of america appears to flip on firearm promise with loan to remington,the bank pledg in april to stop financ compani that sell militari style firearm to civilian remington made the bushmast rifl use in the sandi hook school massacr
8,business,ex volkswagen ceo charged in u s over emissions cheating scandal,martin winterkorn resign soon after the scandal becam public in
9,business,women describe rampant groping sexual harassment at verizon contracted warehouse,verizon launch an investig soon after huffpost reach out for comment


### TF-IDF

In [9]:
news_vectorizer = TfidfVectorizer(stop_words='english')
news_matrix = news_vectorizer.fit_transform(preproses.short_desc)
print 'There are '+ str(news_matrix.get_shape()[0]) + ' documents over a vocabulary feature space of ' + str(news_matrix.get_shape()[1]) + ' terms.'

tfidf = pd.DataFrame(news_matrix.toarray(), index=[preproses.headline, preproses.category], columns= news_vectorizer.get_feature_names())
tfidf

There are 13650 documents over a vocabulary feature space of 14563 terms.


,,aaaargh,aardvark,aardwolv,aaron,aarrr,ab,abandon,abba,abbey,abbi,...,zooland,zoom,zte,zucchini,zuckerberg,zuckerborg,zumba,zuzu,zwaan,zwigoff
headline,category,,,,,,,,,,,,,,,,,,,,,
u s launches auto import probe china vows to defend its interests,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
starbucks says anyone can now sit in its cafes even without buying anything,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
seattle passes controversial new tax on city s biggest companies to combat housing crisis,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
uber ends forced arbitration in individual cases of sexual assault harassment,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
chili s hit by data breach credit and debit card information compromised,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
how uber silences women after sexual assaults,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
how amazon is holding seattle hostage,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
bank of america appears to flip on firearm promise with loan to remington,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
ex volkswagen ceo charged in u s over emissions cheating scandal,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


### DATA TESTING & TRAINING

In [10]:
training_data=tfidf.iloc[:-350]
training_label=preproses.category.iloc[:-350]
testing_data=tfidf.iloc[-350:]
testing_label=preproses.category.iloc[-350:]
testing_title=preproses.headline.iloc[-350:]

### KNN n=170

In [11]:
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=170, metric='cosine')

#Train the model using the training sets
knn.fit(training_data, training_label)

#Predict the response for test dataset
y_pred = knn.predict(testing_data)
#print(y_pred)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(testing_label, y_pred))

('Accuracy:', 0.64)


In [15]:
#print(y_pred)

In [16]:
tampil2= pd.DataFrame(columns=["headline","category","category_predicted"])
i=1
while i<=350:
    #prediction = knn.predict(testing_data[i-1:i])
    tampil2= tampil2.append({"headline":list(testing_title[i-1:i]), "category":list(testing_label[i-1:i]),"category_predicted":knn.predict(testing_data[i-1:i]) }, ignore_index=True)
    i=i+1

#tampil2.to_csv('A:/PWR/project/SMA/knn.csv')
tampil2.head()

In [12]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(testing_label, y_pred))
print(classification_report(testing_label, y_pred))

[[28  5  2  0  8  7  0]
 [ 4 35  3  1  6  1  0]
 [ 4  4 24  4  2 11  1]
 [ 0  2  1 45  0  2  0]
 [10  2  4  2 27  4  1]
 [ 5  5  4  0  3 33  0]
 [ 2  2  4  5  2  3 32]]
               precision    recall  f1-score   support

     business       0.53      0.56      0.54        50
        crime       0.64      0.70      0.67        50
entertainment       0.57      0.48      0.52        50
 food & drink       0.79      0.90      0.84        50
     politics       0.56      0.54      0.55        50
       sports       0.54      0.66      0.59        50
       travel       0.94      0.64      0.76        50

    micro avg       0.64      0.64      0.64       350
    macro avg       0.65      0.64      0.64       350
 weighted avg       0.65      0.64      0.64       350



### NBC

In [18]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(training_data, training_label)

#### NBC ACCURACY

In [19]:
predicted2 = clf.predict(testing_data)
#print(predicted)
np.mean(predicted2 == testing_label)

0.6685714285714286

In [19]:
#print(predicted2)

In [20]:
# jumlah testing data yg diambil hanya 5, dan melihat tetangganya siapa saja
tampil= pd.DataFrame(columns=["headline","category","category_predicted"])
l=1
while l<=100:
    prediction = clf.predict(testing_data[l-1:l])
    tampil= tampil.append({"headline":list(testing_title[l-1:l]), "category":list(testing_label[l-1:l]),"category_predicted":clf.predict(testing_data[l-1:l]) }, ignore_index=True)
    l=l+1

In [21]:
#tampil.to_csv('A:/PWR/project/SMA/NBC.csv')
tampil.head()

,headline,category,category_predicted
0,[european debt crisis in spain soaring joblessness fuels barter economy],[business],[business]
1,[jamie dimon beware the wall street journal says it is time to break up the big banks],[business],[business]
2,[influencer of the week jeremy lin],[business],[sports]
3,[the disingenuous economics of ron paul],[business],[business]
4,[why inequality matters the housing crisis our justice system capitalism],[business],[politics]


In [22]:
print(confusion_matrix(testing_label, predicted2))
print(classification_report(testing_label, predicted2))

[[32  3  2  0  8  4  1]
 [ 4 37  1  0  6  1  1]
 [ 5  5 26  3  1  3  7]
 [ 1  0  0 46  0  0  3]
 [18  3  0  1 25  1  2]
 [ 6  6  3  1  1 31  2]
 [ 6  1  2  3  1  0 37]]
               precision    recall  f1-score   support

     business       0.44      0.64      0.52        50
        crime       0.67      0.74      0.70        50
entertainment       0.76      0.52      0.62        50
 food & drink       0.85      0.92      0.88        50
     politics       0.60      0.50      0.54        50
       sports       0.78      0.62      0.69        50
       travel       0.70      0.74      0.72        50

    micro avg       0.67      0.67      0.67       350
    macro avg       0.69      0.67      0.67       350
 weighted avg       0.69      0.67      0.67       350



In [24]:
from sklearn.naive_bayes import GaussianNB
clf2 = GaussianNB()
clf2.fit(training_data, training_label)

predicted3 = clf2.predict(testing_data)
#print(predicted)
np.mean(predicted3 == testing_label)

0.4085714285714286